In [29]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['database.sqlite', 'hashes.txt', 'Reviews.csv']


In [30]:
data = pd.read_csv('../input/Reviews.csv')

In [31]:
data.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [32]:
data.shape


(568454, 10)

In [33]:
#to remove the datapoints which are neither positive nor negative
data = data[data['Score']!=3]

In [34]:
data.shape

(525814, 10)

In [35]:
#for score>3, it is 'positive' review and score<3 is 'negative' review
def partition(x):
    if x>3:
        return 'positive'
    else:
        return 'negative'

In [36]:
data['Score'] = data['Score'].apply(partition)

In [37]:
sorted_data = data.sort_values(by='ProductId', axis=0)

In [38]:
final = sorted_data.drop_duplicates(subset={"UserId","Time","Text","ProfileName"})
final['Id'].size/sorted_data['Id'].size

0.6925890143662968

In [39]:
final = final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [112]:
final['Score_n'] = final['Score'].apply(lambda x:1 if x == 'positive' else 0)

In [78]:
final_1 = final[:10000]

In [79]:
import re

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [80]:
final_1['Text'] = final_1['Text'].apply(cleanhtml)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [77]:
import string

def cleanpunc(raw_string):
    #print(raw_string)
    newstr = [char for char in raw_string.split() if char not in string.punctuation]
    #print(newstr)
    newstr = ' '.join(newstr)
    return newstr

In [44]:
final_1['Text'] = final_1['Text'].apply(cleanpunc)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [45]:
final_1.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
150523,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
150505,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
150506,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
150507,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
150508,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [46]:
final_data = final_1

In [47]:
from sklearn.model_selection import train_test_split

In [48]:
X_temp, X_test, y_temp, y_test = train_test_split(final_data['Text'], final_data['Score'], test_size= 0.2)

In [49]:
X_train, X_cv, y_train, y_cv = train_test_split(X_temp, y_temp, test_size = 0.2)

In [50]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_cv.shape)
print(y_cv.shape)

(6400,)
(2000,)
(6400,)
(2000,)
(1600,)
(1600,)


In [51]:
#bag of words with bigrams
from sklearn.feature_extraction.text import CountVectorizer

In [52]:
#create an object of countvectorizer with bigrams. 
count_vect = CountVectorizer(ngram_range=(1,2))

In [53]:
#fit transform the training values to train the model. transform the test and cv values so that there is no data leak
final_bigram_train = count_vect.fit_transform(X_train.values)
final_bigram_test = count_vect.transform(X_test.values)
final_bigram_cv = count_vect.transform(X_cv.values)

In [54]:
final_bigram_train.shape

(6400, 208900)

In [55]:
#standardize the data 
from sklearn.preprocessing import StandardScaler

In [56]:
#create an object for StandardScaler
sc = StandardScaler(with_mean=False)

In [57]:
X_train_standardized = sc.fit_transform(final_bigram_train)
X_test_standardized = sc.transform(final_bigram_test)
X_cv_standarized = sc.transform(final_bigram_cv)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [58]:
#to train the model using KNN.
from sklearn.neighbors import KNeighborsClassifier

In [59]:
#create an object of KNN with default values
Knn = KNeighborsClassifier()

In [60]:
Knn.fit(X_train_standardized, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [61]:
y_pred = Knn.predict(X_test_standardized)

In [62]:
#to measure the performance of the simple model
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [63]:
accuracy_score(y_test, y_pred)

0.847

In [64]:
confusion_matrix(y_test, y_pred)

array([[   0,  306],
       [   0, 1694]])

In [65]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       0.00      0.00      0.00       306
    positive       0.85      1.00      0.92      1694

   micro avg       0.85      0.85      0.85      2000
   macro avg       0.42      0.50      0.46      2000
weighted avg       0.72      0.85      0.78      2000



/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [66]:
#data seems highly imbalanced. 
y_train.value_counts()

positive    5363
negative    1037
Name: Score, dtype: int64

In [67]:
y_test.value_counts()

positive    1694
negative     306
Name: Score, dtype: int64

**conclusion:**
> The above model is created using default values of KNN. The data selected was small and was heavily imbalanced. Hence the results are skewed and all are predicted as positive.

In [69]:
#lets consider more data and create one more model. split the data keeping the distribution of the Score same
#across the split

In [81]:
final_1 = final[:30000]
final_1['Text'] = final_1['Text'].apply(cleanhtml)
final_1['Text'] = final_1['Text'].apply(cleanpunc)
final_data = final_1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [82]:
X_temp, X_test, y_temp, y_test = train_test_split(final_data['Text'], final_data['Score'], test_size= 0.2, stratify = final_data['Score'])

In [83]:
X_train, X_cv, y_train, y_cv = train_test_split(X_temp, y_temp, test_size = 0.2, stratify = y_temp)

In [84]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(X_cv.shape)
print(y_cv.shape)

(19200,)
(6000,)
(19200,)
(6000,)
(4800,)
(4800,)


In [85]:
#create an object of countvectorizer with bigrams. 
count_vect_2 = CountVectorizer(ngram_range=(1,2))

In [86]:
#fit transform the training values to train the model. transform the test and cv values so that there is no data leak
final_bigram_train = count_vect_2.fit_transform(X_train.values)
final_bigram_test = count_vect_2.transform(X_test.values)
final_bigram_cv = count_vect_2.transform(X_cv.values)

In [87]:
final_bigram_train.shape

(19200, 444363)

In [88]:
#create an object for StandardScaler
sc = StandardScaler(with_mean=False)

In [89]:
X_train_standardized = sc.fit_transform(final_bigram_train)
X_test_standardized = sc.transform(final_bigram_test)
X_cv_standarized = sc.transform(final_bigram_cv)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [103]:
#create an object of KNN with default values
Knn_2 = KNeighborsClassifier(weights='distance')

In [104]:
Knn_2.fit(X_train_standardized, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='distance')

In [105]:
y_pred = Knn_2.predict(X_test_standardized)

In [106]:
accuracy_score(y_test, y_pred)

0.8516666666666667

In [107]:
confusion_matrix(y_test, y_pred)

array([[  11,  890],
       [   0, 5099]])

In [108]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    negative       1.00      0.01      0.02       901
    positive       0.85      1.00      0.92      5099

   micro avg       0.85      0.85      0.85      6000
   macro avg       0.93      0.51      0.47      6000
weighted avg       0.87      0.85      0.79      6000



**Conclusion:**
>The data was increased 3 times. The data imbalance is causing the model to predict all data to one class only.

In [110]:
from sklearn.model_selection import cross_val_score

In [116]:
final_1 = final[:50000]
final_1['Text'] = final_1['Text'].apply(cleanhtml)
final_1['Text'] = final_1['Text'].apply(cleanpunc)
final_data = final_1

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [117]:
X_train, X_test, y_train, y_test =  train_test_split(final_data['Text'], final_data['Score_n'], test_size = 0.25, stratify = final_data['Score_n'])

In [119]:
#create an object of countvectorizer with bigrams. 
count_vect_3 = CountVectorizer(ngram_range=(1,2))

In [120]:
#fit transform the training values to train the model. transform the test and cv values so that there is no data leak
final_bigram_train = count_vect_3.fit_transform(X_train.values)
final_bigram_test = count_vect_3.transform(X_test.values)

In [121]:
#create an object for StandardScaler
sc = StandardScaler(with_mean=False)

In [122]:
X_train_standardized = sc.fit_transform(final_bigram_train)
X_test_standardized = sc.transform(final_bigram_test)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [132]:
k = [5,11,25, 49,75,99]
cv_score = []
for i in k:
    knn3 = KNeighborsClassifier(n_neighbors=i,weights='distance')
    sc = cross_val_score(knn3, X = X_train_standardized,y = y_train,scoring = 'precision',cv = 5)
    cv_score.append(sc.mean())
    

In [142]:
cv_score = np.array(cv_score)

In [143]:
maxv = cv_score.max()

In [145]:
opt_index = cv_score.argmax()

In [146]:
k[opt_index]

5

In [147]:
knn_4 = KNeighborsClassifier(n_neighbors=5, weights='distance')
knn_4.fit(X_train_standardized, y_train)
y_pred = knn_4.predict(X_test_standardized)
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.85168
[[   11  1854]
 [    0 10635]]


In [148]:
#the K=5 value was used and the result was skewed due to imbalanced data. Using precision as score did not make the model better

**Part 2**
>lets apply logistic regression for the data and understand the performance 

In [149]:
from sklearn.linear_model import LogisticRegression

In [150]:
#creating the logistic regression object with default values
lrmodel = LogisticRegression()

In [151]:
lrmodel.fit(X_train_standardized, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [152]:
y_pred = lrmodel.predict(X_test_standardized)

In [153]:
accuracy_score(y_test, y_pred)

0.89392

In [154]:
confusion_matrix(y_test, y_pred)

array([[  602,  1263],
       [   63, 10572]])

In [155]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.32      0.48      1865
           1       0.89      0.99      0.94     10635

   micro avg       0.89      0.89      0.89     12500
   macro avg       0.90      0.66      0.71     12500
weighted avg       0.90      0.89      0.87     12500



Remark
The logistic regression gives good result with accuracy of 89% . It has good precision but has poor values for recall and f1 score

In [159]:
#lets improve the model by varying the hyperparameter.

In [168]:
C = [1,10, 20,50,70,100]
cv_scores = []
for c in C:
    lrmodel_2 = LogisticRegression(C=c)
    sc = cross_val_score(lrmodel_2, X = X_train_standardized, y = y_train, cv=5, scoring = 'f1_macro')
    cv_scores.append(sc.mean())
    

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [169]:
cv_scores

[0.6253481263962938,
 0.6349818222907931,
 0.6371629974306074,
 0.637480753899925,
 0.6376101515535507,
 0.6377052984260014]

In [170]:
#maximum score is for C = 100

In [171]:
lrmodel_3 = LogisticRegression(C=100)

In [172]:
lrmodel_3.fit(X_train_standardized, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [173]:
y_pred = lrmodel_3.predict(X_test_standardized)

In [174]:
accuracy_score(y_test, y_pred)

0.8964

In [175]:
confusion_matrix(y_test, y_pred)

array([[  650,  1215],
       [   80, 10555]])

In [176]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.35      0.50      1865
           1       0.90      0.99      0.94     10635

   micro avg       0.90      0.90      0.90     12500
   macro avg       0.89      0.67      0.72     12500
weighted avg       0.90      0.90      0.88     12500



In [177]:
#the model is improved than the previous model

In [178]:
y_train.value_counts()

1    31905
0     5595
Name: Score_n, dtype: int64

In [ ]:
#different class weights are provided to see if the prediction improved

In [208]:
lrmodel_4 = LogisticRegression(C = 100, class_weight={1:1,0:3})

In [209]:
lrmodel_4.fit(X_train_standardized, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=100, class_weight={1: 1, 0: 3}, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [210]:
y_pred = lrmodel_4.predict(X_test_standardized)

In [211]:
accuracy_score(y_test, y_pred)

0.8972

In [212]:
confusion_matrix(y_test, y_pred)

array([[  662,  1203],
       [   82, 10553]])

In [213]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.35      0.51      1865
           1       0.90      0.99      0.94     10635

   micro avg       0.90      0.90      0.90     12500
   macro avg       0.89      0.67      0.73     12500
weighted avg       0.90      0.90      0.88     12500



In [216]:
#it is seen that the confusion matrix prediction improved for the weight 1:3